In [1]:
import numpy as np
from numpy.linalg import norm
from time import time
from sys import stdout

In [19]:
V = np.array([
    [1, 2, 3,2,1],  # Usuario 1
    [4, 5, 6,3,1],  # Usuario 2
    [7, 8, 9,4,1],
        # Usuario 3

])
num_users, num_movies = V.shape
num_features = 2  # Número de características latentes

# Inicializar W y H aleatoriamente
np.random.seed(0)
Winit = np.random.rand(num_users, num_features)
Hinit = np.random.rand(num_features, num_movies)

W = Winit
H = Hinit

gradW = np.dot(W, np.dot(H, H.T)) - np.dot(V, H.T)
gradH = np.dot(W.T, np.dot(W, H)) - np.dot(W.T, V)
initgrad = norm(np.r_[gradW, gradH.T])
gradW[np.logical_or(gradW < 0, W > 0)]
gradH[np.logical_or(gradH < 0, H > 0)]
projnorm = norm(np.r_[gradW[np.logical_or(gradW < 0, W > 0)],
                              gradH[np.logical_or(gradH < 0, H > 0)]])
print(projnorm)
np.r_[gradW[np.logical_or(gradW < 0, W > 0)],
                              gradH[np.logical_or(gradH < 0, H > 0)]]
print(gradW[np.logical_or(gradW < 0, W > 0)])
print(gradH[np.logical_or(gradH < 0, H > 0)])
print(np.r_[gradW[np.logical_or(gradW < 0, W > 0)],
                              gradH[np.logical_or(gradH < 0, H > 0)]])



34.227000678221465
[ -3.96028505  -2.6710249  -11.37169165  -8.96719343 -18.9415048
 -15.26634796]
[-5.03009863 -6.18305864 -7.34200192 -4.20625956 -0.82035628 -6.33256738
 -7.73883041 -9.13509107 -5.18018263 -1.01175158]
[ -3.96028505  -2.6710249  -11.37169165  -8.96719343 -18.9415048
 -15.26634796  -5.03009863  -6.18305864  -7.34200192  -4.20625956
  -0.82035628  -6.33256738  -7.73883041  -9.13509107  -5.18018263
  -1.01175158]


In [10]:
def nmf(V, Winit, Hinit, tol, maxiter):
    """
    Perform Non-negative Matrix Factorization (NMF) using projected gradients.

    Parameters:
    V : numpy.ndarray
        Input matrix to factor.
    Winit : numpy.ndarray
        Initial factor matrix W.
    Hinit : numpy.ndarray
        Initial factor matrix H.
    tol : float
        Tolerance for the stopping condition.
    timelimit : float
        Time limit for the computation.
    maxiter : int
        Maximum number of iterations.

    Returns:
    W : numpy.ndarray
        Factor matrix W after NMF.
    H : numpy.ndarray
        Factor matrix H after NMF.
    """
    W = Winit
    H = Hinit
    initt = time()

    gradW = np.dot(W, np.dot(H, H.T)) - np.dot(V, H.T)
    gradH = np.dot(W.T, np.dot(W, H)) - np.dot(W.T, V)
    initgrad = norm(np.r_[gradW, gradH.T])
    print('Init gradient norm %f' % initgrad)

    tolW = max(0.001, tol) * initgrad
    tolH = tolW

    for iter in range(1, maxiter):
        # Stopping condition
        projnorm = norm(np.r_[gradW[np.logical_or(gradW < 0, W > 0)],
                             gradH[np.logical_or(gradH < 0, H > 0)]])
        if projnorm < tol * initgrad :
            break

        (W, gradW, iterW) = nlssubprob(V.T, H.T, W.T, tolW, 1000)
        W = W.T
        gradW = gradW.T

        if iterW == 1:
            tolW = 0.1 * tolW

        (H, gradH, iterH) = nlssubprob(V, W, H, tolH, 1000)
        if iterH == 1:
            tolH = 0.1 * tolH

        if iter % 10 == 0:
            stdout.write('.')

    print('\nIter = %d Final proj-grad norm %f' % (iter, projnorm))
    return (W, H)

In [7]:
def nlssubprob(V, W, Hinit, tol, maxiter):
    """
    Solve the non-negative least squares subproblem.

    Parameters:
    V : numpy.ndarray
        Constant matrix.
    W : numpy.ndarray
        Constant matrix.
    Hinit : numpy.ndarray
        Initial solution matrix H.
    tol : float
        Stopping tolerance.
    maxiter : int
        Maximum number of iterations.

    Returns:
    H : numpy.ndarray
        Updated solution matrix H.
    grad : numpy.ndarray
        Gradient of the objective function.
    iter : int
        Number of iterations used.
    """
    H = Hinit
    WtV = np.dot(W.T, V)
    WtW = np.dot(W.T, W)

    alpha = 1 #incial size
    beta = 0.1 #controla el ajuste de alpha
    for iter in range(1, maxiter):
        grad = np.dot(WtW, H) - WtV
        projgrad = norm(grad[np.logical_or(grad < 0, H > 0)])
        if projgrad < tol:
            break

        # Search for step size
        for inner_iter in range(1, 20):
            Hn = H - alpha * grad
            Hn = np.where(Hn > 0, Hn, 0)
            d = Hn - H
            gradd = np.sum(grad * d)
            dQd = np.sum(np.dot(WtW, d) * d)
            suff_decr = 0.99 * gradd + 0.5 * dQd < 0
            
            if inner_iter == 1:
                decr_alpha = not suff_decr
                Hp = H
            
            if decr_alpha:
                if suff_decr:
                    H = Hn
                    break
                else:
                    alpha = alpha * beta
            else:
                if not suff_decr or np.all(Hp == Hn):
                    H = Hp
                    break
                else:
                    alpha = alpha / beta
                    Hp = Hn

    if iter == maxiter:
        print('Max_iter reached')
    
    return (H, grad, iter)

In [8]:
v = np.array([
    [5, 0, 3, 0, 4],  # Usuario 1
    [1, 1, 0, 0, 0],  # Usuario 2
    [0, 5, 1, 0, 0],  # Usuario 3
    [0, 5, 5, 0, 4],  # Usuario 4
    [4, 5, 3, 0, 2],  # Usuario 5
    [1, 0, 5, 0, 0],  # Usuario 6
    [4, 3, 0, 0, 5]   # Usuario 7
])

In [12]:
num_users, num_movies = v.shape
num_features = 5  # Número de características latentes

# Inicializar W y H aleatoriamente
np.random.seed(0)
Winit = np.random.rand(num_users, num_features)
Hinit = np.random.rand(num_features, num_movies)

tol = 0.001
maxiter = 100   # Máximo de iteraciones

(wo, ho) = nmf(v, Winit, Hinit, tol, maxiter)

Init gradient norm 27.047931

Iter = 10 Final proj-grad norm 0.003434


In [6]:
print('W')
print(wo)

W
[[0.         0.         4.67962614 2.17279207 0.        ]
 [0.         0.         0.55157786 0.08979478 0.69991414]
 [0.         0.12409943 0.         0.59648708 3.44727642]
 [0.         5.75018596 0.         0.32487042 2.03825948]
 [0.         0.         2.91763834 2.26272117 3.48309367]
 [0.         0.0992832  0.         3.49362342 0.        ]
 [0.         0.47458177 4.92236678 0.         1.96179916]]


In [7]:
print('H')
print(ho)

H
[[6.17635497e-01 6.12095723e-01 6.16933997e-01 9.43748079e-01
  6.81820299e-01]
 [0.00000000e+00 3.60318690e-01 7.86459218e-01 0.00000000e+00
  7.00949830e-01]
 [8.85423564e-01 7.00714629e-04 0.00000000e+00 0.00000000e+00
  8.65007566e-01]
 [3.73439384e-01 0.00000000e+00 1.38545645e+00 0.00000000e+00
  0.00000000e+00]
 [1.87015709e-02 1.43659180e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]


In [8]:
print("Matrix aprox")
print(np.dot(wo, ho))

Matrix aprox
[[4.95485739e+00 3.27908250e-03 3.01030878e+00 0.00000000e+00
  4.04791202e+00]
 [5.35002431e-01 1.00587741e+00 1.24406753e-01 0.00000000e+00
  4.77119018e-01]
 [2.87221253e-01 4.99704440e+00 9.24006009e-01 0.00000000e+00
  8.69874711e-02]
 [1.59438064e-01 5.00004634e+00 4.97238057e+00 0.00000000e+00
  4.03059187e+00]
 [3.49347426e+00 5.00582825e+00 3.13490163e+00 0.00000000e+00
  2.52377923e+00]
 [1.30465658e+00 3.57735911e-02 4.91834527e+00 0.00000000e+00
  6.95925393e-02]
 [4.39506826e+00 2.99275445e+00 3.73239211e-01 0.00000000e+00
  4.59054252e+00]]
